Testing operations_1_-0-

In [1]:
from pyspark import SparkContext

sc = SparkContext(appName="appName") # < напишите код здесь >
weather_entry = sc.parallelize(['2009-01-01', 15.1,26.1]) # < напишите код здесь >
print(weather_entry.take(3))

['2009-01-01', 15.1, 26.1]


In [2]:
# Не забудьте остановить SparkContext после использования
sc.stop()

In [3]:
from pyspark.sql import SparkSession
 
APP_NAME = 'sampleApp'
spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
# Вывести свойства объекта spark
print("AppName:", spark.sparkContext.appName)
print("Master:", spark.sparkContext.master)
print("Version:", spark.version)
# Другие свойства, которые вы хотите вывести...

AppName: sampleApp
Master: local[*]
Version: 3.4.1


In [4]:
spark.stop()

In [13]:
# from pyspark.sql import SparkSession
# # Вводим Spark Master IP, который выдаст нам скрипт 
# SPARK_MASTER_IP = '172.18.0.2' 
# spark = SparkSession.builder.appName("pyspark-taxi-forecasting") \
#     .master(f"spark://{SPARK_MASTER_IP}:7077") \
#     .config("spark.executor.cores", 1) \
#     .config("spark.task.cpus", 1) \
#     .getOrCreate()
## .config("spark.executor.memory", "1g") \

In [5]:
import os

# Получаем значение переменной окружения SPARK_MASTER_IP
spark_master_ip = os.getenv('SPARK_MASTER_IP')

In [8]:
# Если переменная окружения установлена, используем ее для настройки Spark Session
if spark_master_ip:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.appName("pyspark-taxi-forecasting") \
        .master(f"spark://{spark_master_ip}:7077") \
        .config("spark.executor.cores", "1") \
        .config("spark.task.cpus", 1) \
        .getOrCreate()
    print(f"Spark Master IP: {spark_master_ip}")
else:
    print("Переменная окружения SPARK_MASTER_IP не установлена")

Spark Master IP: 172.26.0.2


In [9]:
# Вывести свойства объекта spark
print("AppName:", spark.sparkContext.appName)
print("Master:", spark.sparkContext.master)
print("Version:", spark.version)

AppName: pyspark-taxi-forecasting
Master: spark://172.26.0.2:7077
Version: 3.4.1


In [10]:
%%time
taxi = spark.read.load('/work/data/Taxi_Trips_-_2023.csv', 
                       format='csv', header='true', inferSchema='true')

CPU times: user 8.83 ms, sys: 3.24 ms, total: 12.1 ms
Wall time: 28.2 s


In [11]:
%%time
# Вывести схему данных
taxi.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

In [12]:
# Отобразить первые несколько строк DataFrame
# taxi.show()

In [13]:
import pandas as pd

In [14]:
table_descr = pd.read_csv('/work/data/table_descr.csv',sep=';')

In [15]:
table_descr

,Column Name,Description,Type,Описание,Тип,Unnamed: 5
0,Trip ID,A unique identifier for the trip.,Plain Text,Уникальный идентификатор поездки,Обычный текст,Ошибка:510
1,Taxi ID,A unique identifier for the taxi.,Plain Text,Уникальный идентификатор такси,Обычный текст,NaN
2,Trip Start Timestamp,When the trip started rounded to the nearest 1...,Date & Time,Время начала поездки округленное до ближайших...,Дата и время,NaN
3,Trip End Timestamp,When the trip ended rounded to the nearest 15 ...,Date & Time,Время окончания поездки округленное до ближай...,Дата и время,NaN
4,Trip Seconds,Time of the trip in seconds.,Number,Продолжительность поездки в секундах,Число,NaN
5,Trip Miles,Distance of the trip in miles.,Number,Расстояние поездки в милях,Число,NaN
6,Pickup Census Tract,The Census Tract where the trip began. For pri...,Plain Text,Код района по переписи населения где началас...,Обычный текст,NaN
7,Dropoff Census Tract,The Census Tract where the trip ended. For pri...,Plain Text,Код района по переписи населения где заверши...,Обычный текст,NaN
8,Pickup Community Area,The Community Area where the trip began. This ...,Number,Номер коммунальной области где началась поез...,Число,NaN
9,Dropoff Community Area,The Community Area where the trip ended. This ...,Number,Номер коммунальной области где завершилась п...,Число,NaN


In [16]:
taxi[["Trip ID", "Taxi ID", "Trip Miles","Trip Start Timestamp","Trip End Timestamp","Trip Seconds","Trip Miles","Pickup Census Tract"]].show(5)

+--------------------+--------------------+----------+--------------------+--------------------+------------+----------+-------------------+
|             Trip ID|             Taxi ID|Trip Miles|Trip Start Timestamp|  Trip End Timestamp|Trip Seconds|Trip Miles|Pickup Census Tract|
+--------------------+--------------------+----------+--------------------+--------------------+------------+----------+-------------------+
|0fca59218b1168827...|e2c349c7cbb608d55...|      4.82|01/01/2023 12:00:...|01/01/2023 12:15:...|        1037|      4.82|               null|
|1e539d6e7501164c6...|4ab7a7510c1ebcc9b...|     16.63|01/01/2023 12:00:...|01/01/2023 12:15:...|        1341|     16.63|               null|
|2b3c5200439d51f62...|8c76eb82f069c0731...|      3.84|01/01/2023 12:00:...|01/01/2023 12:15:...|         844|      3.84|               null|
|45b2ea39cfff64d61...|a688de71e9eb70603...|      0.63|01/01/2023 12:00:...|01/01/2023 12:00:...|         361|      0.63|               null|
|464df6aaaf97

In [17]:
taxi[["Trip ID", "Dropoff Census Tract", "Pickup Community Area","Dropoff Community Area","Trip End Timestamp","Trip Total","Trip Miles","Company"]].show(5)

+--------------------+--------------------+---------------------+----------------------+--------------------+----------+----------+--------------------+
|             Trip ID|Dropoff Census Tract|Pickup Community Area|Dropoff Community Area|  Trip End Timestamp|Trip Total|Trip Miles|             Company|
+--------------------+--------------------+---------------------+----------------------+--------------------+----------+----------+--------------------+
|0fca59218b1168827...|                null|                    6|                    32|01/01/2023 12:15:...|      19.5|      4.82|Taxicab Insurance...|
|1e539d6e7501164c6...|                null|                   76|                     8|01/01/2023 12:15:...|      53.0|     16.63|            Sun Taxi|
|2b3c5200439d51f62...|                null|                   24|                     8|01/01/2023 12:15:...|     20.17|      3.84|            Sun Taxi|
|45b2ea39cfff64d61...|                null|                   32|                 

In [19]:
taxi[["Trip ID", "Pickup Centroid Latitude", "Pickup Centroid Longitude","Pickup Centroid Location",
      "Dropoff Centroid Latitude","Dropoff Centroid Longitude","Dropoff Centroid  Location"]].show(5)

+--------------------+------------------------+-------------------------+------------------------+-------------------------+--------------------------+--------------------------+
|             Trip ID|Pickup Centroid Latitude|Pickup Centroid Longitude|Pickup Centroid Location|Dropoff Centroid Latitude|Dropoff Centroid Longitude|Dropoff Centroid  Location|
+--------------------+------------------------+-------------------------+------------------------+-------------------------+--------------------------+--------------------------+
|0fca59218b1168827...|            41.944226601|            -87.655998182|    POINT (-87.655998...|             41.878865584|             -87.625192142|      POINT (-87.625192...|
|1e539d6e7501164c6...|            41.980264315|            -87.913624596|    POINT (-87.913624...|             41.899602111|             -87.633308037|      POINT (-87.633308...|
|2b3c5200439d51f62...|            41.901206994|            -87.676355989|    POINT (-87.676355...|       

In [20]:
# print(taxi.describe().show())
# Запустите метод describe() на вашем PySpark DataFrame (например, taxi)
describe_result = taxi.describe().toPandas()

In [21]:
describe_result

,summary,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,count,3783730,3783730,3783730,3783682,3783012,3783717,1650232,1617623,3615963,...,3778327,3778327,3783730,3783730,3617351,3617351,3617351,3441935,3441935,3441935
1,mean,None,None,None,None,1235.3278387697421,6.471233194766421,1.7031501647110167E10,1.7031414613682808E10,35.03091652209937,...,2.2418538840073934,27.371134848836103,None,None,41.90200067412448,-87.69906175284666,None,41.89419484402875,-87.66236339478645,None
2,stddev,None,None,None,None,1736.5661018295189,7.593310441212217,373503.06470608304,344107.34525214223,26.0787201470195,...,19.47903838213312,37.03373571024758,None,None,0.06251123711511891,0.11213128488994713,None,0.05656711663584079,0.07345541720850367,None
3,min,0000012deb83dbb55726d5a75c374197d0641fa0,00110971c7c4a7173fcf93f49a22d6b9b0a02c27c4b9f8...,01/01/2023 01:00:00 AM,01/01/2023 01:00:00 AM,0,0.0,17031010100,17031010100,1,...,0.0,0.0,Cash,2733 - 74600 Benny Jona,41.651921576,-87.913624596,POINT (-87.5313862567 41.7204632831),41.660136051,-87.913624596,POINT (-87.5349029012 41.707311449)
4,max,fffffe03acfa1552c98fad12d73ff0aca70a5c2a,ffd231d2536b9463d888cfbb42f36d543b37d22d96a6dd...,08/01/2023 12:00:00 AM,10/17/2022 10:00:00 AM,86340,945.4,17031980100,17031980100,77,...,9446.65,9999.75,Unknown,U Taxicab,42.021223593,-87.531386257,POINT (-87.913624596 41.9802643146),42.021223593,-87.534902901,POINT (-87.913624596 41.9802643146)


In [26]:
describe_result = pd.read_csv('/work/2023_describe.csv',sep=',')

In [27]:
print(describe_result.transpose())

                                    0                   1              2  \
summary                         count                mean         stddev   
Trip ID                       3783730                 NaN            NaN   
Taxi ID                       3783730                 NaN            NaN   
Trip Start Timestamp          3783730                 NaN            NaN   
Trip End Timestamp            3783682                 NaN            NaN   
Trip Seconds                3783012.0         1235.327839    1736.566102   
Trip Miles                  3783717.0            6.471233        7.59331   
Pickup Census Tract         1650232.0  17031501647.110168  373503.064706   
Dropoff Census Tract        1617623.0  17031414613.682808  344107.345252   
Pickup Community Area       3615963.0           35.030917       26.07872   
Dropoff Community Area      3419046.0           26.097846      20.917892   
Fare                        3778327.0            21.99552      22.233947   
Tips        

In [99]:
# output_file_path = '/work/2023_describe.csv'
# describe_result.to_csv(output_file_path, index=False) #transpose().

In [28]:
# taxi.registerTempTable("taxi")
"""
/usr/local/spark/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)
"""
taxi.createOrReplaceTempView("taxi")

In [29]:
%%time
# вам нужно использовать обратные кавычки ` ` (backticks) или просто имя столбца без кавычек
print(spark.sql("SELECT COUNT(*) FROM taxi WHERE `Trip Seconds` < 300").show())

+--------+
|count(1)|
+--------+
|  489757|
+--------+

None
CPU times: user 6.41 ms, sys: 0 ns, total: 6.41 ms
Wall time: 7.53 s


In [30]:
from pyspark.sql.functions import col, dayofweek, to_timestamp, rand, substring

In [31]:
def random_sample_dataframe(dataframe, percentage):
    # Генерируем случайные числа от 0 до 1 и фильтруем строки
    taxis = dataframe.filter(rand() < percentage)
    
    return taxis

In [32]:
taxis = random_sample_dataframe(taxi, 0.05)

In [33]:
taxis[["Trip ID", "Trip Start Timestamp"]].show(5)

+--------------------+--------------------+
|             Trip ID|Trip Start Timestamp|
+--------------------+--------------------+
|2b3c5200439d51f62...|01/01/2023 12:00:...|
|6e26e8341d032293c...|01/01/2023 12:00:...|
|e56ff16af8b0b5234...|01/01/2023 12:00:...|
|1117f21cfce35b2bd...|01/01/2023 12:15:...|
|8a3a48dd3283e3c9b...|01/01/2023 12:15:...|
+--------------------+--------------------+
only showing top 5 rows



In [33]:
def determine_date_format(dataframe):
    # Выбираем первые два символа из столбца "Trip Start Timestamp"
    first_two_chars = dataframe.select(substring("Trip Start Timestamp", 1, 2))
    
    # Подсчитываем уникальные значения первых двух символов
    unique_values = first_two_chars.distinct().count()
    
    # Определяем маску на основе количества уникальных значений
    date_format = "dd/MM/yyyy" if unique_values >= 13 else "MM/dd/yyyy"
    
    return date_format, unique_values

In [79]:
%%time
# Пример использования:
date_format, unique_values = determine_date_format(taxis)
print("Дата формат:", date_format,", количество уникальных значений в 1-х двух знаках:",unique_values)

Дата формат: MM/dd/yyyy , количество уникальных значений в 1-х двух знаках: 8
CPU times: user 5.09 ms, sys: 1.6 ms, total: 6.69 ms
Wall time: 6.53 s


In [80]:
taxis.select(substring("Trip Start Timestamp", 1, 2)).distinct().show()

+-------------------------------------+
|substring(Trip Start Timestamp, 1, 2)|
+-------------------------------------+
|                                   03|
|                                   04|
|                                   07|
|                                   02|
|                                   05|
|                                   06|
|                                   01|
|                                   08|
+-------------------------------------+



In [81]:
taxis[["Trip Start Timestamp"]].show(n=15, truncate=False)

+----------------------+
|Trip Start Timestamp  |
+----------------------+
|01/01/2023 12:00:00 AM|
|01/01/2023 12:00:00 AM|
|01/01/2023 12:00:00 AM|
|01/01/2023 12:00:00 AM|
|01/01/2023 12:00:00 AM|
|01/01/2023 12:00:00 AM|
|01/01/2023 12:00:00 AM|
|01/01/2023 12:00:00 AM|
|01/01/2023 12:15:00 AM|
|01/01/2023 12:15:00 AM|
|01/01/2023 12:15:00 AM|
|01/01/2023 12:15:00 AM|
|01/01/2023 12:15:00 AM|
|01/01/2023 12:15:00 AM|
|01/01/2023 12:15:00 AM|
+----------------------+
only showing top 15 rows



In [82]:
taxis.select(to_timestamp(taxis["Trip Start Timestamp"], 'MM/dd/yyyy hh:mm:ss a').alias("Trip Start Timestamp")).show(15)

+--------------------+
|Trip Start Timestamp|
+--------------------+
| 2023-01-01 00:00:00|
| 2023-01-01 00:00:00|
| 2023-01-01 00:00:00|
| 2023-01-01 00:00:00|
| 2023-01-01 00:00:00|
| 2023-01-01 00:00:00|
| 2023-01-01 00:00:00|
| 2023-01-01 00:00:00|
| 2023-01-01 00:15:00|
| 2023-01-01 00:15:00|
| 2023-01-01 00:15:00|
| 2023-01-01 00:15:00|
| 2023-01-01 00:15:00|
| 2023-01-01 00:15:00|
| 2023-01-01 00:15:00|
+--------------------+
only showing top 15 rows



In [85]:
taxis = taxis.withColumn("Trip Start Timestamp", to_timestamp(taxis["Trip Start Timestamp"], 'MM/dd/yyyy hh:mm:ss a'))

In [86]:
taxis[["Trip Start Timestamp"]].show(n=15, truncate=False)

+--------------------+
|Trip Start Timestamp|
+--------------------+
|2023-01-01 00:00:00 |
|2023-01-01 00:00:00 |
|2023-01-01 00:00:00 |
|2023-01-01 00:00:00 |
|2023-01-01 00:00:00 |
|2023-01-01 00:00:00 |
|2023-01-01 00:00:00 |
|2023-01-01 00:00:00 |
|2023-01-01 00:15:00 |
|2023-01-01 00:15:00 |
|2023-01-01 00:15:00 |
|2023-01-01 00:15:00 |
|2023-01-01 00:15:00 |
|2023-01-01 00:15:00 |
|2023-01-01 00:15:00 |
+--------------------+
only showing top 15 rows



In [87]:
taxis = taxis.withColumn("DayOfWeek", dayofweek(col("Trip Start Timestamp")))

In [93]:
%%time
# taxis.createOrReplaceTempView("taxis")
q = \
"""
SELECT
    DayOfWeek,
    AVG(`Trip Seconds`)
FROM taxis
GROUP BY DayOfWeek
"""
print(spark.sql(q).show(10))

+---------+------------------+
|DayOfWeek| avg(Trip Seconds)|
+---------+------------------+
|        1|1251.8656300217413|
|        6|1245.3464070073985|
|        3|1208.4795715916075|
|        5|1268.5626337898955|
|        4|1223.8157303753105|
|        7|1179.3079529999331|
|        2|1237.2134771972103|
+---------+------------------+

None
CPU times: user 5.69 ms, sys: 1.78 ms, total: 7.47 ms
Wall time: 8.98 s


Вывод %%time в Jupyter Notebook содержит информацию о времени выполнения кода в двух разных контекстах:

    CPU times: Это время, которое ваш код фактически проводит в CPU (Central Processing Unit) или процессоре. Он разбивается на две части:
        user: Это время, в течение которого CPU выполняет пользовательский код (ваш код).
        sys: Это время, которое CPU тратит на выполнение системных операций, таких как ввод/вывод данных.

    Wall time: Это общее время, которое прошло с начала выполнения кода до его завершения. Это включает в себя не только вычислительное время, но и время, которое может быть затрачено на ожидание ввода/вывода, блокировку потоков и другие факторы, которые могут вызвать задержки в выполнении кода.

В вашем конкретном выводе:

    user составляет 10.5 миллисекунд (0.0105 секунд).
    sys составляет 2.63 миллисекунд (0.00263 секунд).
    total (сумма user и sys) равно 13.2 миллисекунд (0.0132 секунд).

Это время, которое фактически затратилось на выполнение кода на процессоре.

    Wall time составляет 25.3 секунды. Это общее время с момента запуска ячейки до ее завершения, включая все вычисления и возможные задержки.

Обратите внимание, что Wall time (время на стенке) обычно больше, чем сумма user и sys, потому что оно включает в себя также время, которое может быть затрачено на ожидание ввода/вывода или другие факторы, которые могут замедлить выполнение кода.

In [89]:
taxis.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: timestamp (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centro

In [90]:
taxis[["Trip ID", "Trip Start Timestamp","DayOfWeek"]].show(5)

+--------------------+--------------------+---------+
|             Trip ID|Trip Start Timestamp|DayOfWeek|
+--------------------+--------------------+---------+
|1e539d6e7501164c6...| 2023-01-01 00:00:00|        1|
|121060d23b4c3cf77...| 2023-01-01 00:00:00|        1|
|26d8ad56a2c363618...| 2023-01-01 00:00:00|        1|
|3e23ed6f78b4f284e...| 2023-01-01 00:00:00|        1|
|3f07cb261574b2047...| 2023-01-01 00:00:00|        1|
+--------------------+--------------------+---------+
only showing top 5 rows

